# HERMES 3.0 UK Biobank Phenotyping
#### Nicholas Sunderland - Mar 2025
#### nicholas.sunderland@bristol.ac.uk

## **Warning** - these scripts have changed 14/03/25

## Setup
First open an R JupytrLab session on the UK Biobank RAP with standard settings. 

Open the terminal app and clone the heRmes Github repository into `/opt/notebooks` by running: 

`git clone https://github.com/nicksunderland/heRmes.git`

This will put the **heRmes** directory at `/opt/notebooks/heRmes`. 

First we will need to extract the necessary data. The data extraction script will pull data from the databases into your project directory storgae. The script is located at: 

`/opt/notebooks/heRmes/scripts/extract_ukbb_data.ipynb`

Ensure this runs and wait for the table extraction jobs to finish, resulting in a new folder in your project called `hermes3_data` filled with data tables. 

The HERMES 3.0 phenotyping script (this script) is located at:

`/opt/notebooks/heRmes/scripts/hermes3_phenotyping_ukbb_rap.ipynb`

Running this script will process the phenotypes, resulting in a phenotype file at called ``hermes3_data/hermes3_phenotypes.tsv.gz` and a counts summary at `hermes3_data/hermes3_phenotype_summary.tsv`.

You need to set the project and record IDs below to your own project.

* **Project ID**: the `Project ID` is available from the 'Settings' tab after clicking on your project in the UK Biobak RAP.  
* **Record ID**: the record or dataset `ID` is available from the 'Manage' tab, where you select the checkbox to the left of your dispensed `.dataset` (e.g. app81499_20241105095754.dataset) which then displays the `ID` in the information pane on the right.

In [1]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
recordid  <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"

### Libraries

In [2]:
library(glue)
library(data.table)
library(yaml)
setwd("/opt/notebooks")
source("/opt/notebooks/heRmes/R/ukbb_extraction_utils.R")

## Read in extracted data

In [3]:
# get the extract config
config <- read_yaml("/opt/notebooks/heRmes/scripts/ukbb_extraction_config.yml")

# take the required tables for this phenotyping
config <- config[c("participant", "hesin", "hes_diag", "hes_oper")]

# function to 

# read data 
data <- list()
for (i in seq_along(config)) {
    
    file_path <- file.path("/mnt/project", config[[i]][["output"]])
    
    if (!file.exists(file_path)) {
        cat(glue("Error:\nFile {file_path} not found, ",
                 "check the Monitor tab for the status of the Table-exporter ",
                 "and the 'hermes_data' folder. If this has finished try ",
                 "launching another Notebook session/instance (I'm not sure why ",
                 "the mounted /mnt/project/ file structure doesn't refresh when ",
                 "files are added externally."), sep="\n")
        stop("file not found error")
    }
    
    n <- names(config)[i]
    cat(glue('...{n}: {file_path}\n'), sep="\n")
    flush.console()
    data[[n]] <- fread(file_path)
    
    #print(typeof(config[[i]][["columns"]]))
    rename_ukbb_cols(data[[n]], col_config=config[[i]][["columns"]])
    
}

lapply(data, head, n = 5)

...participant: /mnt/project/hermes3_data/data_participant.tsv
...hesin: /mnt/project/hermes3_data/data_hesin.tsv
...hes_diag: /mnt/project/hermes3_data/data_hesin_diag.tsv
...hes_oper: /mnt/project/hermes3_data/data_hesin_oper.tsv


$participant
       eid assessment_date_1 assessment_date_2 assessment_date_3
     <int>            <IDat>            <IDat>            <IDat>
1: 1000074        2009-03-20              <NA>              <NA>
2: 1000194        2009-06-27              <NA>        2019-07-03
3: 1000258        2008-05-02              <NA>              <NA>
4: 1000280        2008-04-28              <NA>        2016-09-02
5: 1000299        2009-06-13              <NA>              <NA>
   assessment_date_4 assessment_age_1 assessment_age_2 assessment_age_3
              <IDat>            <int>            <int>            <int>
1:              <NA>               42               NA               NA
2:              <NA>               60               NA               70
3:              <NA>               56               NA               NA
4:              <NA>               44               NA               52
5:              <NA>               45               NA               NA
   assessment_age_4 reason_lost_fu   sex sample_collection_date_1
              <int>          <int> <int>                   <POSc>
1:               NA             NA     0      2009-03-20 16:49:16
2:               NA             NA     1      2009-06-27 11:07:49
3:               NA             NA     1      2008-05-02 18:19:27
4:               NA             NA     1      2008-04-28 18:36:55
5:               NA             NA     0      2009-06-13 15:54:13
   sample_collection_date_2 sample_collection_date_3 sample_collection_date_4
                     <POSc>                   <POSc>                   <POSc>
1:                     <NA>                     <NA>                     <NA>
2:                     <NA>      2019-07-03 15:17:56                     <NA>
3:                     <NA>                     <NA>                     <NA>
4:                     <NA>      2016-09-02 10:37:12                     <NA>
5:                     <NA>                     <NA>                     <NA>
   weight_1 weight_2 weight_3 weight_4 height_1 height_2 height_3 height_4
      <num>    <num>    <num>    <num>    <num>    <num>    <num>    <num>
1:     67.3       NA       NA       NA      162       NA       NA       NA
2:     76.0       NA     76.1       NA      165       NA    162.9       NA
3:     85.5       NA       NA       NA      182       NA       NA       NA
4:     84.5       NA     82.9       NA      175       NA    175.0       NA
5:     49.6       NA       NA       NA      160       NA       NA       NA
     bmi_1 bmi_2   bmi_3 bmi_4 ethnicity_1 ethnicity_2 ethnicity_3 ethnicity_4
     <num> <num>   <num> <num>       <int>       <int>       <int>       <int>
1: 25.6440    NA      NA    NA        1001          NA          NA          NA
2: 27.9155    NA 28.6776    NA        1001          NA          NA          NA
3: 25.8121    NA      NA    NA        1001          NA          NA          NA
4: 27.5918    NA 27.0694    NA        1001          NA        1001          NA
5: 19.3750    NA      NA    NA        1001          NA          NA          NA
   genetic_sex genetic_ethnicity      pc1      pc2       pc3        pc4
         <int>             <int>    <num>    <num>     <num>      <num>
1:           0                 1 -11.2546 3.867470  1.694130 -1.8339000
2:           1                 1 -11.9589 2.585230 -2.532330 -0.6596200
3:           1                NA -12.7644 0.704641  0.550128  3.1239700
4:           1                 1 -12.3484 4.465790 -1.581400  0.0703759
5:           0                 1 -12.3153 2.796960  0.408848  0.9702480
        pc5 self_rep_ill_1 self_rep_ill_2 self_rep_ill_3 self_rep_ill_4
      <num>          <int>          <int>          <int>          <int>
1: -6.58044           1351             NA             NA             NA
2: -4.94291           1086             NA             NA             NA
3: -7.03787           1065           1265           1465             NA
4: -1.89994           1436             NA             NA             NA
5: -2.

## Data processing

### Read in the heRmes codes
ICD-9/10 coding is provided but we need to add the self reported codes from the UK-BB too.

In [4]:
codes <- fread(file.path("/opt/notebooks", "heRmes", "inst", "extdata", "hermes_3_codes", "hermes_3_codes.tsv"))
self_reported_codes <- list(
  list(name = "Heart Failure",                      code = "1076", code_type = "ukbb_self_reported_illness"),
  list(name = "Myocardial infarction",              code = "1075", code_type = "ukbb_self_reported_illness"),
  list(name = "Hypertrophic cardiomyopathy",        code = "1588", code_type = "ukbb_self_reported_illness"),
  list(name = "Coronary artery bypass grafting",    code = "1095", code_type = "ukbb_self_reported_procedure"),
  list(name = "Percutaneous coronary intervention", code = "1070", code_type = "ukbb_self_reported_procedure")
)
codes <- rbind(codes,
               data.table(Concept     = paste0(sapply(self_reported_codes, function(x) x$name), " Self Reported"),
                          Code        = sapply(self_reported_codes, function(x) x$code),
                          Source      = sapply(self_reported_codes, function(x) x$code_type),
                          Description = sapply(self_reported_codes, function(x) x$name)))
codes[, `:=`(code      = Code,
             code_type = fcase(Source=="ICD10", "icd10",
                               Source=="ICD9",  "icd9",
                               Source=="OPCS4", "opcs4",
                               Source=="ukbb_self_reported_illness", "ukbb_self_reported_illness",
                               Source=="ukbb_self_reported_procedure", "ukbb_self_reported_procedure"))]
codes <- codes[!is.na(code_type)] 
head(codes)

Concept,Code,Source,Description,code,code_type
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Heart Failure,I110,ICD10,Hypertensive heart disease with heart failure,I110,icd10
Heart Failure,I130,ICD10,"Hypertensive heart and chronic kidney disease with heart failure and stage 1 through stage 4 chronic kidney disease, or unspecified chronic kidney disease",I130,icd10
Heart Failure,I132,ICD10,"Hypertensive heart and chronic kidney disease with heart failure and with stage 5 chronic kidney disease, or end stage renal disease",I132,icd10
Heart Failure,I50,ICD10,Heart failure,I50,icd10
Heart Failure,I500,ICD10,Congestive heart failure,I500,icd10
Heart Failure,I501,ICD10,"Left ventricular failure, unspecified",I501,icd10


### Clean up the cohort data

In [6]:
ethnicity_codes <- list(
  white                 = 1,
  british               = 1001,
  white_black_caribbean = 2001,
  indian                = 3001,
  caribbean             = 4001,
  mixed                 = 2,
  irish                 = 1002,
  white_black_african   = 2002,
  pakistani             = 3002,
  african               = 4002,
  asian_or_asian_british= 3,
  any_other_white       = 1003,
  white_asian           = 2003,
  bangladeshi           = 3003,
  any_other_black       = 4003,
  black_or_black_british= 4,
  any_other_mixed       = 2004,
  any_other_asian       = 3004,
  chinese               = 5,
  other_ethnic_group    = 6)

data$participant[, ethnicity := fcoalesce(.SD), .SDcols = names(data$participant)[grepl("^ethnicity_[0-9]$", names(data$participant))]]

data$demog <- data$participant[, 
    list(eid               = eid,
         reason_lost_fu    = reason_lost_fu,
         assessment_age    = as.integer(assessment_age_1),
         sex               = factor(sex, levels = 0:1, labels = c("female", "male")),
         ethnicity         = factor(ethnicity, levels = unlist(ethnicity_codes), labels = names(ethnicity_codes)),
         ethnicity_group   = factor(sub("([0-9])00[0-9]", "\\1", ethnicity), levels = unlist(ethnicity_codes), labels = names(ethnicity_codes)),
         genetic_sex       = factor(genetic_sex, levels = 0:1, labels = c("female", "male")),
         genetic_ethnicity = factor(genetic_ethnicity, levels = 1, labels = c("caucasian")), 
         pc1               = pc1,
         pc2               = pc2,
         pc3               = pc3,
         pc4               = pc4,
         pc5               = pc5)]

### Self-report illness codes to long

In [7]:
self_rep_code_cols <- grep("self_rep_ill_[0-9]+",      names(data$participant), value = TRUE)
self_rep_year_cols <- grep("self_rep_ill_year_[0-9]+", names(data$participant), value = TRUE)
data$participant[, (self_rep_code_cols) := lapply(.SD, as.character), .SDcols = self_rep_code_cols]
data$participant[, (self_rep_year_cols) := lapply(.SD, as.numeric),   .SDcols = self_rep_year_cols]
data$self_illness <- data.table::melt(data$participant,
                                      id.vars = "eid",
                                      measure = patterns("self_rep_ill_[0-9]+", "self_rep_ill_year_[0-9]+"),
                                      variable.name = "element",
                                      value.name = c("code", "year"),
                                      na.rm = TRUE)
data$self_illness <- data$self_illness[year != -1 & year != -3] # unknown / prefer not to answer
data$self_illness[, `:=`(date      = lubridate::ymd(paste0(as.character(floor(year)), "-01-01")) + lubridate::days(as.integer(365.25 * (year - floor(year)))),
                         year      = NULL,
                         element   = NULL,
                         code      = as.character(code),
                         code_type = "ukbb_self_reported_illness")]

# check self report illness table
stopifnot("unable to parse dates for self-reported illness codes" = all(!is.na(data$self_illness$date)))
stopifnot("are you sure something happened before 1900?" = all(data$self_illness$date > as.Date("1900-01-01")))

### Self-report procedure codes to long

In [8]:
self_rep_proc_code_cols <- grep("self_rep_proc_[0-9]+",      names(data$participant), value = TRUE)
self_rep_proc_year_cols <- grep("self_rep_proc_year_[0-9]+", names(data$participant), value = TRUE)
data$participant[, (self_rep_proc_code_cols) := lapply(.SD, as.character), .SDcols = self_rep_proc_code_cols]
data$participant[, (self_rep_proc_year_cols) := lapply(.SD, as.numeric),   .SDcols = self_rep_proc_year_cols]
data$self_oper <- data.table::melt(data$participant,
                                   id.vars = "eid",
                                   measure = patterns("self_rep_proc_[0-9]+", "self_rep_proc_year_[0-9]+"),
                                   variable.name = "element",
                                   value.name = c("code", "year"),
                                   na.rm = TRUE)
data$self_oper <- data$self_oper[year != -1 & year != -3] # unknown / prefer not to answer
data$self_oper[, `:=`(date      = lubridate::ymd(paste0(as.character(floor(year)), "-01-01")) + lubridate::days(as.integer(365.25 * (year - floor(year)))),
                      year      = NULL,
                      element   = NULL,
                      code      = as.character(code),
                      code_type = "ukbb_self_reported_procedure")]

# check self report illness table
stopifnot("unable to parse dates for self-reported procedure codes" = all(!is.na(data$self_oper$date)))
stopifnot("are you sure something happened before 1900?" = all(data$self_oper$date > as.Date("1900-01-01")))

### Inpatient diagnosis codes

In [9]:
data$hesin[is.na(epistart) | epistart == "", epistart := admidate]
data$hes_diag[data$hesin, date := as.Date(i.epistart), on = c("eid", "ins_index")]
data$hes_diag[diag_icd9 == "", diag_icd9 := NA_character_]
data$hes_diag[diag_icd10 == "", diag_icd10 := NA_character_]
data$hes_diag <- data.table::melt(data$hes_diag,
                                  id.vars = c("eid", "date"),
                                  measure.vars  = c("diag_icd9", "diag_icd10"),
                                  variable.name = "code_type",
                                  value.name = "code",
                                  na.rm = TRUE)
data$hes_diag[, code_type := data.table::fcase(code_type == "diag_icd9", "icd9",
                                               code_type == "diag_icd10", "icd10")]

### Inpatient procedure codes

In [10]:
data$hes_oper[data$hesin, date := as.Date(i.epistart), on = c("eid", "ins_index")]
data$hes_oper[oper3 == "", oper3 := NA_character_]
data$hes_oper[oper4 == "", oper4 := NA_character_]
data$hes_oper <- data.table::melt(data$hes_oper,
                                  id.vars = c("eid", "date"),
                                  measure.vars  = c("oper3", "oper4"),
                                  variable.name = "code_type",
                                  value.name = "code",
                                  na.rm = TRUE)
data$hes_oper[, code_type := data.table::fcase(code_type == "oper3", "opcs3",
                                               code_type == "oper4", "opcs4")]

Warning message in melt.data.table(data$hes_oper, id.vars = c("eid", "date"), measure.vars = c("oper3", :
“'measure.vars' [oper3, oper4, ...] are not all of the same type. By order of hierarchy, the molten data value column will be of type 'character'. All measure variables not of type 'character' will be coerced too. Check DETAILS in ?melt.data.table for more on coercion.”


### Combine all codes
Keep only unique codes per individuals at the code's first occurance.

In [11]:
combined <- rbind(data$self_illness, data$self_oper, data$hes_diag, data$hes_oper)
combined <- codes[combined, on = c("code" = "code", "code_type" = "code_type"), allow.cartesian = TRUE]
combined <- combined[!is.na(Concept)]
combined <- combined[combined[, .I[which.min(date)], by = c("eid", "Concept")]$V1]

### Annotate the cohort with the codes

In [12]:
cohort <- data$demog
concepts <- unique(codes$Concept)
for (g in concepts) {

  col_name <- tolower(gsub(" ", "_", gsub("[()]","",g)))
  cohort[combined[Concept == g], paste0(col_name, c("", "_first_date")) := list(TRUE, as.Date(i.date)), on = "eid"]
  cohort[is.na(get(col_name)), (col_name) := FALSE]

}

### Remove withdrawals

In [13]:
cat(glue('{cohort[reason_lost_fu==5, .N]} withdrawals to remove'), sep="\n")
cohort <- cohort[is.na(reason_lost_fu) | reason_lost_fu!=5] # 5 - Participant has withdrawn consent for future linkage

158 withdrawals to remove


## Run phenotyping

In [ ]:
# any ischaemic ICD codes
ischaemic_cols <- c("myocardial_infarction", "coronary_artery_bypass_grafting", "percutaneous_coronary_intervention", "thrombolysis_coronary", "ischaemic_cardiomyopathy")
cohort[, ischaemic := rowSums(.SD) > 0, .SDcols = ischaemic_cols]
cohort[, ischaemic_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(ischaemic_cols, "_first_date")]

# combined NICM ICD codes
nicm_cols <- c("dilated_cardiomyopathy", "dilated_cardiomyopathy_associated_with", "left_ventricular_systolic_dysfunction")
cohort[, nicm_comb := rowSums(.SD) > 0, .SDcols = nicm_cols]
cohort[, nicm_comb_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(nicm_cols, "_first_date")]

# any ischaemic self reported codes
self_isch_cols <- c("myocardial_infarction_self_reported", "coronary_artery_bypass_grafting_self_reported", "percutaneous_coronary_intervention_self_reported")
cohort[, self_isch := rowSums(.SD) > 0, .SDcols = self_isch_cols]
cohort[, self_isch_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_isch_cols, "_first_date")]

# any self reported HF codes
self_hf_col = c("heart_failure_self_reported")
cohort[, self_hf := rowSums(.SD) > 0, .SDcols = self_hf_col]
cohort[, self_hf_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_hf_col, "_first_date")]

# any self reported HCM codes
self_hcm_col = c("hypertrophic_cardiomyopathy_self_reported")
cohort[, self_hcm := rowSums(.SD) > 0, .SDcols = self_hcm_col]
cohort[, self_hcm_first_date := do.call(pmin, c(.SD, na.rm = TRUE)), .SDcols = paste0(self_hcm_col, "_first_date")]

# HF exclusions
cohort[, hf_exclude := congenital_heart_disease==TRUE |  # all congenital heart disease
                       (heart_failure==FALSE & (self_hf==TRUE | ischaemic==TRUE | self_isch==TRUE)) | # non-HF but with ischaemic ICD history or self reported heart failure or ischaemic history
                       (heart_failure==TRUE  & (ischaemic==TRUE & ischaemic_first_date > heart_failure_first_date))] # HF but with first ischaemic event after the HF diagnosis
# pheno 1
cohort[, pheno1 := congenital_heart_disease==FALSE & heart_failure==TRUE]

# pheno 2
cohort[, pheno2 := hf_exclude==FALSE & heart_failure==TRUE & ischaemic==TRUE]

# pheno 3
cohort[, pheno3 := hf_exclude==FALSE & heart_failure==TRUE & ischaemic==FALSE]

# HF controls
cohort[, hf_control := hf_exclude==FALSE & pheno1==FALSE & pheno2==FALSE & pheno3==FALSE]

# DCM exclusions
cohort[, cm_exclude := congenital_heart_disease==TRUE |  # all congenital heart disease
                       hypertrophic_cardiomyopathy==TRUE | self_hcm==TRUE | # all HCM
                       restrictive_cardiomyopathy==TRUE] # all RCM

# pheno 4
cohort[, pheno4 := cm_exclude==FALSE &
                   !(dilated_cardiomyopathy==TRUE  & (ischaemic==TRUE & ischaemic_first_date <= dilated_cardiomyopathy_first_date)) & # DCM but with first ischaemic event prior to the DCM diagnosis
                   dilated_cardiomyopathy==TRUE]

# pheno 5
cohort[, pheno5 := cm_exclude==FALSE &
         !(nicm_comb==TRUE & (ischaemic==TRUE & ischaemic_first_date <= nicm_comb_first_date)) &  # NICM but with first ischaemic event prior to the NICM diagnosis
         nicm_comb==TRUE]

# CM controls
cohort[, cm_control := cm_exclude==FALSE &
                       pheno4==FALSE &
                       pheno5==FALSE &
                       ischaemic==FALSE &
                       self_isch==FALSE]


# check HF phenotyping
base_cols <- c("eid", "assessment_age", "sex", "ethnicity", "ethnicity_group","genetic_sex", "genetic_ethnicity", paste0("pc",1:5))
sum_cols <- names(cohort)[!names(cohort) %in% base_cols
                          &
                          !grepl("date", names(cohort))]
summary <- data.table (name = c("total", sum_cols), sex = "all", N = c(nrow(cohort), cohort[, .(sapply(.SD, sum)), .SDcols = sum_cols]$V1))
summary <- rbind(summary,
                 data.table(name = rep(sum_cols, 2), cohort[, .(N = sapply(.SD, sum)), .SDcols = sum_cols, by = "sex"]), fill=TRUE)
summary

# write summary
fwrite(dcast(summary, name ~ sex, value.var = "N"),
       file = "hermes3_phenotype_summary.tsv",
       sep  = "\t")

cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/1) = ", sum(summary[name%in%c("hf_exclude",paste0("pheno1"),"hf_control"), N]), "\n")
cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/2/3) = ", sum(summary[name%in%c("hf_exclude",paste0("pheno",2:3),"hf_control"), N]), "\n")
cat("Total =", summary[name=="total",N], "; Sum (exc/crtl/5) = ", sum(summary[name%in%c("cm_exclude",paste0("pheno5"),"cm_control"), N]), "\n")

# write out
fwrite(cohort[, mget(c(base_cols,
                       "congenital_heart_disease", 
                       "congenital_heart_disease_procedure",
                       "hypertrophic_cardiomyopathy", 
                       "hypertrophic_cardiomyopathy_self_reported",
                       paste0("pheno", 1:3), "hf_exclude", "hf_control",
                       paste0("pheno", 4:5), "cm_exclude", "cm_control"))],
       file = "hermes3_phenotypes.tsv.gz",
       sep  = "\t")

name,sex,N
<chr>,<fct>,<int>
total,all,501976
reason_lost_fu,all,NA
heart_failure,all,20156
dilated_cardiomyopathy,all,1597
dilated_cardiomyopathy_associated_with,all,1707
left_ventricular_systolic_dysfunction,all,10373
myocardial_infarction,all,26911
coronary_artery_bypass_grafting,all,7505
percutaneous_coronary_intervention,all,17933


Total = 501976 ; Sum (exc/crtl/1) =  1007346 
Total = 501976 ; Sum (exc/crtl/2/3) =  1003952 
Total = 501976 ; Sum (exc/crtl/5) =  932770 


### Copy output to project

In [15]:
head(cohort)


eid,reason_lost_fu,assessment_age,sex,ethnicity,ethnicity_group,genetic_sex,genetic_ethnicity,pc1,pc2,⋯,self_hcm_first_date,hf_exclude,pheno1,pheno2,pheno3,hf_control,cm_exclude,pheno4,pheno5,cm_control
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,⋯,<date>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1000074,NA,42,female,british,white,female,caucasian,-11.2546,3.867470,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
1000194,NA,60,male,british,white,male,caucasian,-11.9589,2.585230,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
1000258,NA,56,male,british,white,male,NA,-12.7644,0.704641,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
1000280,NA,44,male,british,white,male,caucasian,-12.3484,4.465790,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
1000299,NA,45,female,british,white,female,caucasian,-12.3153,2.796960,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE
1000314,NA,56,female,british,white,female,caucasian,-11.3328,2.678840,⋯,NA,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,TRUE


In [20]:
o <- system("dx upload hermes3_phenotype_summary.tsv hermes3_phenotypes.tsv.gz --destination hermes3_data", intern = TRUE)
cat(o, sep = "\n")

Warning message in system("dx upload hermes3_phenotype_summary.tsv hermes3_phenotypes.tsv.gz --destination hermes3_data", :
“running command 'dx upload hermes3_phenotype_summary.tsv hermes3_phenotypes.tsv.gz --destination hermes3_data' had status 3”


# End